# Industry visualization
This project is all about helping users understand the charactistics of a given industry and understand them in the context of all industries. It mainly uses Prof. Damodaran's data. Only publicly traded companies in the industry are considered. 

In [1]:
# Import required libraries
import pandas as pd
import numpy as np

import sys
import os

sys.path.append("../")
from edgar_utils import ticker_to_cik, get_companywide_concepts, get_financial_item
from adamodaran_utils import *

from pprint import pprint
pd.set_option('display.width', 1000)

In [6]:
data_df = get_ind_multiples(industry_list=['Apparel', 'Air Transport'])
pprint(data_df.head())
pprint(len(data_df))

                    PBV  EV/ Invested Capital  Current PE  Trailing PE  Forward PE  Price/Sales  EV/Sales  EV/EBITDA    EV/EBIT
Industry Name                                                                                                                  
Air Transport  2.941324              1.778373   28.715790    17.981924   12.441417     0.550876  1.022212   7.465027  17.630171
Apparel        3.136963              2.474594   18.531965    28.054987   21.845891     1.087106  1.413761   9.215376  14.635936
2


In [16]:
# Risk factors
data_df = get_adamodar_file("betas")
average_beta_colname = data_df.filter(regex="^Average").columns[0] # Extract the column name for Average unlevered Beta corrected for cash 
data_df.columns = data_df.columns.str.strip()
risk_df = data_df[['Number of firms', 'Beta', 'D/E Ratio', average_beta_colname]].rename(columns = {average_beta_colname:'UBCC'})

data_df = get_adamodar_file("wacc")
cols_of_interest = ['Cost of Equity', 'After-tax Cost of Debt', 'Cost of Capital']
risk_df[cols_of_interest] = data_df[cols_of_interest]

data_df = get_adamodar_file("pedata")
risk_df['% of Money Losing firms'] = data_df['% of Money Losing firms (Trailing)']
pprint(risk_df.head())


                   Number of firms  Beta  D/E Ratio      UBCC  Cost of Equity  After-tax Cost of Debt  Cost of Capital  % of Money Losing firms
Industry Name                                                                                                                                  
Advertising                     54  1.34     0.2620  1.119543        0.103705                0.048075         0.092157                 0.777778
Aerospace/Defense               67  0.90     0.2279  0.995942        0.084837                0.041475         0.076788                 0.432836
Air Transport                   24  1.24     1.0683  0.783627        0.099319                0.048075         0.072851                 0.708333
Apparel                         37  0.99     0.4589  0.962945        0.088692                0.043350         0.074430                 0.621622
Auto & Truck                    34  1.62     0.2240  1.205260        0.115747                0.048075         0.103365                 0

In [43]:
# Fundamentals
data_df = get_adamodar_file("DollarUS")
cols_of_interest = ['Revenues ($ millions)', 'Gross Profit ($ millions)', 'EBITDA ($ millions)', 'EBIT (Operating Income) ($ millions)', 'Net Income ( $ millions)']
rename_dict = {'Revenues ($ millions)': 'Reveue', 
               'Gross Profit ($ millions)': 'Gross Profit', 
               'EBITDA ($ millions)':'EBITDA', 
               'EBIT (Operating Income) ($ millions)': 'EBIT', 
               'Net Income ( $ millions)': 'Net Income'} 
fundamentals_df = data_df[cols_of_interest].rename(columns = rename_dict)
print(fundamentals_df.head())

                       Reveue  Gross Profit     EBITDA          EBIT  Net Income
Industry  Name                                                                  
Advertising         42768.949     12793.402   7340.853   4810.743013    1282.101
Aerospace/Defense  494053.030     84251.971  56673.314  37834.835120   21577.625
Air Transport      235530.200     58787.060  29240.129  12869.809560    5209.720
Apparel             67822.858     36812.565  10270.019   6518.405518    2021.261
Auto & Truck       486262.586     54009.651  36450.790  15604.095120   18343.993


In [ ]:
# Efficiency
data_df = get_adamodar_file("wcdata")

efficiency_df = pd.DataFrame(index=data_df.index)
efficiency_df['DSO'] = np.round(data_df['Acc Rec/ Sales']*365)
cogs_ds = fundamentals_df['Reveue'] - fundamentals_df['Gross Profit']
efficiency_df['DSI'] = np.round(data_df['Inventory/Sales']*fundamentals_df['Reveue']*365/cogs_ds)
efficiency_df['DPO'] = np.round(data_df['Acc Pay/ Sales']*fundamentals_df['Reveue']*365/cogs_ds)

data_df = get_adamodar_file("Employee")
cols_of_interest = ['Revenues per Employee  ($)']
efficiency_df[cols_of_interest] = data_df[cols_of_interest]
pprint(efficiency_df.head())

                   Number of firms  Acc Rec/ Sales  Inventory/Sales  Acc Pay/ Sales  Non-cash WC/ Sales
Industry Name                                                                                          
Advertising                     54        0.543894         0.044978        0.537780            0.032308
Aerospace/Defense               67        0.217975         0.287070        0.100070            0.386962
Air Transport                   24        0.038431         0.030239        0.057294            0.010804
Apparel                         37        0.121144         0.189484        0.098481            0.240094
Auto & Truck                    34        0.049562         0.114900        0.149662           -0.019122
---------------------------
Industry  Name
Advertising           29975.547
Aerospace/Defense    409801.059
Air Transport        176743.140
Apparel               31010.293
Auto & Truck         432252.935
dtype: float64
---------------------------
                     DSO  

In [3]:
# Performance measures
data_df = get_adamodar_file("margin")
cols_of_interest = ['Gross Margin', 'Net Margin', 'After-tax Lease & R&D adj Margin', 'EBITDA/Sales','R&D/Sales', 'SG&A/ Sales']
performance_df = data_df[cols_of_interest]

data_df = get_adamodar_file("EVA")
cols_of_interest = ['ROE', '(ROE - COE)', 'ROC', '(ROC - WACC)', 'EVA (US $ millions)']
rename_dict = {'EVA (US $ millions)': 'EVA'}
temp_df = data_df[cols_of_interest].rename(columns = rename_dict)
performance_df[temp_df.columns] = temp_df[temp_df.columns]
print(performance_df.head())
print("---------------------------")


                   Gross Margin  Net Margin  After-tax Lease & R&D adj Margin  EBITDA/Sales  R&D/Sales  SG&A/ Sales       ROE  (ROE - COE)       ROC  (ROC - WACC)           EVA
Industry Name                                                                                                                                                                   
Advertising            0.299128    0.029977                          0.105336      0.137176   0.021491     0.135697  0.118527     0.014822  0.349131      0.256974   3315.934043
Aerospace/Defense      0.170532    0.043675                          0.070255      0.112247   0.040800     0.071603  0.119410     0.034573  0.140254      0.063465  15706.213973
Air Transport          0.249595    0.022119                          0.049501      0.110310   0.003806     0.037423  0.132719     0.033400  0.084764      0.011913   1638.535109
Apparel                0.542775    0.029802                          0.088433      0.117890   0.001135     0.453209

In [80]:
data_df = get_adamodar_file("histgr")
cols_of_interest = ['CAGR in Net Income- Last 5 years', 'CAGR in Revenues- Last 5 years', 'Expected Growth in Revenues - Next 2 years', 'Expected Growth in Revenues - Next 5 years']
rename_dict = {'CAGR in Net Income- Last 5 years': 'Net Income CAGR (past 5y)',
               'CAGR in Revenues- Last 5 years':'Revenue CAGR (past 5y)',
               'Expected Growth in Revenues - Next 2 years': 'Revenue CAGR (next 2y)',
               'Expected Growth in Revenues - Next 5 years': 'Revenue CAGR (next 5y)'}
demand_df = data_df[cols_of_interest].rename(columns = rename_dict)

pprint(demand_df.head())

                   Net Income CAGR (past 5y)  Revenue CAGR (past 5y)  Revenue CAGR (next 2y)  Revenue CAGR (next 5y)
Industry Name                                                                                                       
Advertising                         0.174275               -0.016177                0.090254                0.072864
Aerospace/Defense                   0.034865                0.082055                0.117000                0.124394
Air Transport                      -0.035889                0.026369                0.382927                0.530995
Apparel                            -0.035919                0.069669                0.001860                0.023173
Auto & Truck                        0.083675                0.108952                0.966848                0.351925


In [17]:
# Multiples
data_df = get_adamodar_file("pbvdata")
cols_of_interest = ['PBV', 'EV/ Invested Capital']
multiples_df = data_df[cols_of_interest]

data_df = get_adamodar_file("pedata")
cols_of_interest = ['Current PE', 'Trailing PE', 'Forward PE']
multiples_df[cols_of_interest] = data_df[cols_of_interest]

data_df = get_adamodar_file("psdata")
cols_of_interest = ['Price/Sales', 'EV/Sales']
multiples_df[cols_of_interest] = data_df[cols_of_interest]

data_df = get_adamodar_file("vebitda")
cols_of_interest = [('Only positive EBITDA firms', 'EV/EBITDA'), ('Only positive EBITDA firms', 'EV/EBITDA')]

multiples_df['EV/EBITDA'] = data_df[('Only positive EBITDA firms', 'EV/EBITDA')]
multiples_df['EV/EBIT'] = data_df[('Only positive EBITDA firms', 'EV/EBIT')]
pprint(multiples_df.head())
# print("---------------------------")


                        PBV  EV/ Invested Capital  Current PE  Trailing PE  Forward PE  Price/Sales  EV/Sales  EV/EBITDA     EV/EBIT
Industry Name                                                                                                                       
Advertising        6.976657              8.294943  514.369727   291.270179   73.711931     2.334685  2.753032  15.557573   24.270405
Aerospace/Defense  6.825577              5.383927   39.163388    87.417779   35.067972     2.206060  2.604180  17.237956   28.387133
Air Transport      2.941324              1.778373   28.715790    17.981924   12.441417     0.550876  1.022212   7.465027   17.630171
Apparel            3.136963              2.474594   18.531965    28.054987   21.845891     1.087106  1.413761   9.215376   14.635936
Auto & Truck       6.854352              3.112769   31.424978    27.282768   29.802240     2.938045  3.469955  37.443235  106.522279
